# Hindcasts

When doing a long hindcast simulation is preferable to split the process in several steps and use hotstarting for compliting the task.


- Requires:

    - The data folder. See [README](README.md) in this folder.

In [ ]:
#to use the full width of the browser window
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
# Not neccessarily needed. Just to check version. Must be >= 0.6
import pyposeidon
pyposeidon.__version__

In [ ]:
import pyposeidon.model as pm

### Graphics

In [ ]:
import pyposeidon.utils.pplot # initialize matplolib accessor

In [ ]:
#optional
#pyposeidon.utils.pplot.__init__(dark_background=True) # set plt style for pplot graphics below

In [ ]:
import pyposeidon.utils.hplot # initialize pyviz accessor

- In order to have within the Notebook the executable binaries installed in the conda env of the Kernel we add the corresponding path to the env `PATH`. This is not needed when we are running from python.

In [ ]:
#Create a folder to save the output
import os
if not os.path.exists('test'):
            os.makedirs('test')

In [ ]:
import numpy
cpath = numpy.__path__[0].split('/lib/')[0] # get the current kernel path

os.environ['PATH'] += os.pathsep + cpath + '/bin' # add to PATH

In [ ]:
import geopandas as gp
# use cartopy to get coastlines
import cartopy.feature as cf

Without loss of generality we are using the natural earth's intermediate resolution dataset as coastlines

In [ ]:
cr='i'

coast = cf.NaturalEarthFeature(
    category='physical',
    name='land',
    scale='{}m'.format({'l':110, 'i':50, 'h':10}[cr]))

In [ ]:
import geopandas as gp
ne_i = gp.GeoDataFrame(geometry = [x for x in coast.geometries()])

## Setup

The minimum information required is an initial model and corresponding run. We set this one up. For more info see the [MODEL](./MODEL.ipynb) Notebook.


In [ ]:
#define in a dictionary the properties of the model..
model={'solver_name':'schism',
     'geometry':{'lon_min' : -25,'lon_max' : -12.,'lat_min' : 56.,'lat_max' : 74.}, 
     'coastlines':ne_i,
     'mesh_generator' : 'jigsaw', # set grid generator 
     'start_date':'2011-01-1 0:0:0',
     'time_frame':'12h',
     'meteo_source' : './data/era5.grib', #path to meteo files
     'dem_source' : './data/dem.nc', #path to dem file
     'rpath':'./test/hindcast/20110101.00/', #location of calc folder
     'parameters':{'dt':400, 'rnday':.5, 'nhot':1, 'ihot':0,'nspool':9, 'ihfskip':36, 'nhot_write':108 } # set param.in components
    }

* Note that the parameters arguments will override relevant settings

## Initialize

In [ ]:
#initialize a model
b = pm.set(**model)

In [ ]:
#Execute
b.execute()

## Setup a hindcast

Let's assume we want to split the hindcast on a `12h` splits. Doing a hindcast doesn't, in general, entail overlaping meteo forcing. However when more than one meteo file is needed one needs to be consistent into aligning the dates accordingly. There is an extra argument that provides that functionality (see documentation). 

We setup the hindcast process as

In [ ]:
from pyposeidon.utils import cast
import pandas as pd
import datetime

In [ ]:
# creating a time sequence of the runs
start_date = pd.to_datetime("2011-1-1 0:0:0")
end_date = pd.to_datetime("2011-1-2 0:0:0")
date_list = pd.date_range(start_date, end_date, freq="12h")
date_list

In [ ]:
# creating a sequence of folder to store the runs. In this case we name them after the date attribute.
# NOTE that the first folder is the fisrt run already perfomed!!
rpaths = ['./test/hindcast/' + datetime.datetime.strftime(x, "%Y%m%d.%H") + "/" for x in date_list]
rpaths

So now we can execute the cast sequencially as

In [ ]:
for l in range(len(rpaths) - 1):

    fc = cast.set(solver_name='schism',
            model=b,
            ppath=rpaths[l],
            cpath=rpaths[l + 1],
            meteo='./data/era5.grib',
            sdate=date_list[l + 1],
                ) 
    fc.run(execute=True)

## Output

The output data can be combined using the data utility as

In [ ]:
from pyposeidon.utils import data

In [ ]:
d = data.get_output(solver_name='schism', folders=rpaths)

In [ ]:
d.Dataset

In [ ]:
d.folders